# Good Title - half of success!!! 

In this model try to analyse text part - Title of book. We will receive model which analysing Title and can say about future rating of new book. Let's focus on success of book and devide balls: 0-4 will consider like "fail", 5-10 -it is "success".

In [1]:
from sklearn import  cross_validation, grid_search, linear_model, metrics

import numpy as np
import pandas as pd

import mysql.connector
from mysql.connector import errorcode

C:\Users\Semenov.R\AppData\Local\Continuum\anaconda3\envs\yourenvname\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Semenov.R\AppData\Local\Continuum\anaconda3\envs\yourenvname\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cnx = mysql.connector.connect(user='sena', password='qWERtYhGfDsA12#', host='10.3.99.94', database='bookstore')
cursor = cnx.cursor()

query = ("select Title, case when `Book-Rating`>=5 then 1 else 0 end Raiting "
"from factrating f "
"left join DimUsers du on (f.userid=du.userid) "
"left join Dimcity dct on (du.cityid=dct.cityid) "
"left join DimState ds on (dct.stateid = ds.stateid) "
"left join Dimcountry dc on (ds.countryid=dc.countryid) "
"left join DimBooks db on (db.ISBN=f.isbn) "
"; "         
)
 
cursor.execute(query)

from pandas import DataFrame
df = DataFrame(cursor.fetchall())
df.columns = ('Title',  'Raiting')


cursor.close()
cnx.close()

In [4]:
print df.shape
print df.Raiting.value_counts()

(1149769, 2)
0    735535
1    414234
Name: Raiting, dtype: int64


In [5]:
df.isnull().values.any()

False

In [6]:
TRAIN_SIZE = 0.7 # Разделение данных на обучающую и контрольную части в пропорции 70/30%
from sklearn.cross_validation import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(df.Title.values, df.Raiting.values)

Transform text using standart module which devides text on world. Model will analyse impact word on ball 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train = vect.fit_transform(texts_train)
X_test = vect.transform(texts_test)

C:\Users\Semenov.R\AppData\Local\Continuum\anaconda3\envs\yourenvname\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Build model and analyse result of prediction 

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(X_train, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.6394624325518451
0.5785258608222096


In [9]:
X_train.shape

(862326, 71550)

Detail analyse show that : there are a lot of properties in model. For improving result we will drop properties using Lasso:

In [10]:
clf = LogisticRegression(C=0.15, penalty='l1')
clf.fit(X_train, y_train)
print np.sum(np.abs(clf.coef_) > 1e-4)
print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

1551
0.6410523129803126
0.5694967890244387


In [11]:
from sklearn.linear_model import RandomizedLogisticRegression
rlg = RandomizedLogisticRegression(C=0.13)
rlg.fit(X_train, y_train)

C:\Users\Semenov.R\AppData\Local\Continuum\anaconda3\envs\yourenvname\lib\site-packages\sklearn\utils\deprecation.py:57: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Semenov.R\AppData\Local\Continuum\anaconda3\envs\yourenvname\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


RandomizedLogisticRegression(C=0.13, fit_intercept=True, memory=None,
               n_jobs=1, n_resampling=200, normalize=True,
               pre_dispatch='3*n_jobs', random_state=None,
               sample_fraction=0.75, scaling=0.5, selection_threshold=0.25,
               tol=0.001, verbose=False)

In [12]:
X_train_lasso = X_train[:, rlg.scores_ > 0]
X_test_lasso = X_test[:, rlg.scores_ > 0]

In [13]:
clf = LogisticRegression(C=1)
clf.fit(X_train_lasso, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test_lasso))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_lasso)[:, 1])

0.6403947913151476
0.5209011619074084
